# Parabolic Focus Determination Data Acquisition

This notebook executes a parabolic focus data acquisition for a specific filter/grating configuration. It performs fine sampling of z-axis hexapod translations and takes images at each hexapod position. The analysis will be done in a separate notebook. 

Also contained in this notebook is a CWFS execution, that, in some cases, might be useful as a comparison with the results of the parabolic focus . 

## Set up

### Import libraries

In [1]:
import sys
import asyncio
import time
import os

import numpy as np
import logging 
import yaml
import matplotlib.pyplot as plt
import astropy
from datetime import date

from lsst.ts import salobj
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign
from lsst.ts.observatory.control.utils import RotType

from lsst.ts.idl.enums.Script import ScriptState

from lsst.ts.observing.utilities.decorated_logger import DecoratedLogger

### Setting up logger

In [2]:
logger = DecoratedLogger.get_decorated_logger()
logger.level = logging.DEBUG

### Getting unique index for script

In [3]:
logger.info(f'Your UID is {os.getuid()}')
index = os.getuid() * 10 + np.random.randint(0, 9)

logger.info(f'The generated index is {index}')

### Instantiate CWFS Script

In many cases, it is desirable to perform a CWFS focus sequence for comparison with the parabolic focus. Therefore, LatissCWFSAlign script is instantiated, to provide the option to the user after the parabolic focus data acquisition is done. 

In [ ]:
script = LatissCWFSAlign(index=index, remotes=True)  # this essentially calls the init method
await script.start_task

### Forward ATCS and LATISS

In [ ]:
atcs = script.atcs
latiss = script.latiss

### Set up script log level

In [ ]:
script.log.level = logging.DEBUG

---
## Parabolic Focus Data Acquisition

### Configuration

#### ATSpectrograph configuration

Get available instrument configurations:

In [ ]:
inst_setup = await script.latiss.get_available_instrument_setup()
logger.info(f'Filters are: {inst_setup[0]},\nGratings are: {inst_setup[1]}')

Declare the configuration of the instrument for the focus determination sequence. Both variables are strings. 

In [ ]:
filter_to_focus = 'SDSSr'
grating_to_focus = 'holo4_003'

#### ATAOS corrections

ATAOS corrections must be enabled for this test. In the cell below, ATAOS corrections will be enabled, in case they were not. 

In [ ]:
test = await script.atcs.rem.ataos.evt_correctionEnabled.aget()
if not (test.m1 and test.hexapod and test.atspectrograph):
    cmd = await script.atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)
    logger.info(f'ATAOS corrections enabled: {cmd.result}')

### Slew to target

#### Declare target 

Declare target name

In [ ]:
target = 'HD 110304'

or query for a target at a user-defined azimuth and elevation

In [ ]:
target = await script.atcs.find_target(az=120, el = 60, mag_limit=8)

#### Slew to the defined target

In [ ]:
await script.atcs.slew_object(target, rot_type=RotType.PhysicalSky)

Uncomment the following line to take a snapshot to verify the target is positioned as expected 

In [ ]:
# await script.latiss.take_engtest(2, filter=filter_to_focus, grating=grating_to_focus)

To offset the telescope and center the source (if required) uncomment the following line. <br>
Offsets are in detector X/Y coordinates and in arcseconds. 

In [ ]:
# await script.atcs.offset_xy(x=20, y=20)

### Acquiring Data

Acquire data in steps of `z_offset_step` in the range of `z_offset_start` to `z_offset_end` relative to the current focus position.

In [ ]:
original_focus_offset = await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()
logger.info(f'Original focus offset is \n {original_focus_offset}')

In [ ]:
script.log.info(f'START -- Focus Determination {filter_to_focus}+{grating_to_focus} \n'
                    f'Time UTC -- {astropy.time.Time(astropy.time.Time.now())}') 

z_offset_start = -0.1 # mm
z_offset_step = 0.025 # mm
z_offset_end = -z_offset_start

steps = np.arange(z_offset_start, z_offset_end + 0.01, z_offset_step)
parabolic_focus_data = []

await atcs.rem.ataos.cmd_offset.set_start(z=z_offset_start)
    
await asyncio.sleep(2)
for step in steps:
        
    exposure = await latiss.take_engtest(
        1, 1, filter=filter_to_focus, grating=grating_to_focus, reason='Parabolic_focus_sequence')
    parabolic_focus_data.append(exposure[0])
    logger.info(f'Total z offset = {step:.3f} -- Image expId = {exposure[0]}')
    
    logger.info(f'Current focus offset is \n {await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()}')
    
    await atcs.rem.ataos.cmd_offset.set_start(z=z_offset_step)

# Move back to the original focus offset position
await atcs.rem.ataos.cmd_offset.set_start(z= -(z_offset_end + z_offset_step))
logger.info(f'Back to the original focus offset position \n {await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()}')

# Log results into EFD for later analysis
script.log.info(f'END -- Focus Determination {filter_to_focus}+{grating_to_focus} \n'
                    f'Images expId -- {parabolic_focus_data} \n'
                    f'Time UTC -- {astropy.time.Time(astropy.time.Time.now())}') 

In [ ]:
current_focus_offset = await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()
logger.info(current_focus_offset)

In [ ]:
# Original and current focus offset's user applied should be now equal.  

In [ ]:
assert current_focus_offset.userApplied == original_focus_offset.userApplied

### Stop tracking

If required, then uncomment and use the following cell to stop the telescope from tracking, but you will lose your acquisition.

In [ ]:
# await script.atcs.stop_tracking()

In case it's needed, below is the CWFS focusing execution to compare with the parabolic focus sequence. 

## CWFS Focusing 

### Set up configuration

In [ ]:
configuration = yaml.safe_dump({"filter": filter_to_focus, 
                                "grating": grating_to_focus,
                                "exposure_time": 20,})

The next line is not required the first time the script is run, however, in each additional instance the cell is run, an error will be thrown if it is not included.  
Therefore, it is included here despite being a non-operation in the first instance.  

In [ ]:
script.set_state(ScriptState.UNCONFIGURED)

### Put the ScriptState to CONFIGURED

In [ ]:
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

Set these script parameters to None to verify a re-reduction does not happen of the images.

In [ ]:
script.intra_visit_id = None
script.extra_visit_id = None
script.short_timeout = 10

### Set groupID and launch the script

This sets the same group ID for all exposures taken in the script.

In [ ]:
group_id_data = script.cmd_setGroupId.DataType(
    groupId=astropy.time.Time.now().isot)

await script.do_setGroupId(group_id_data)
await script.arun()

### Stop tracking

If required, then uncomment and use the following cell to stop the telescope from tracking, but you will lose your acquisition.

In [ ]:
# await script.atcs.stop_tracking()